In [1]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 29.5 MB/s 
     |████████████████████████████████| 16.6 MB 40.9 MB/s 
     |████████████████████████████████| 57 kB 6.7 MB/s 
     |████████████████████████████████| 73 kB 2.3 MB/s 
     |████████████████████████████████| 1.2 MB 55.7 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [2]:
!python /content/createDatasets.py

In [3]:
# Create the YAML configuration file

config = '''# config.yaml


## Where the samples will be written
save_data: experiment_1/run/example

# Training files
data:
    corpus_1:
        path_src: experiment_1/src_train.txt
        path_tgt: experiment_1/tgt_train.txt
        transforms: [filtertoolong]
    valid:
        path_src: experiment_1/src_valid.txt
        path_tgt: experiment_1/tgt_valid.txt
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: experiment_1/run/example.vocab.src
tgt_vocab: experiment_1/run/example.vocab.tgt


# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000


# Tokenization options
#src_subword_model: source.model
#tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: experiment_1/run/model

# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 5000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 1234

# Default: 100000 - Train the model to max n steps 
# Increase for large datasets
train_steps: 10000

# Default: 10000 - Run validation after n steps
valid_steps: 1000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 100

decoder_type: transformer
encoder_type: transformer
word_vec_size: 512
feature_vec_size: 512
rnn_size: 512
layers: 5
transformer_ff: 512
heads: 2

accum_count: 4
optim: adam
adam_beta1: 0.9
adam_beta2: 0.998
decay_method: noam
learning_rate: 0.1
max_grad_norm: 0.0



# Tokens per batch, change if out of GPU memory
batch_size: 32

batch_type: tokens
normalization: tokens
dropout: 0.3
label_smoothing: 0.6

max_generator_batches: 2

param_init: 0.0
param_init_glorot: 'true'
position_encoding: 'true'

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [4]:
!onmt_build_vocab -config config.yaml -n_sample 10000 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 14:36:19,669 INFO] Counter vocab from 10000 samples.
[2022-03-26 14:36:19,669 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-03-26 14:36:19,685 INFO] corpus_1's transforms: TransformPipe(FilterTooLongTransform(src_seq_length=200, tgt_seq_length=200))
[2022-03-26 14:36:19,685 INFO] corpus_1's transforms: TransformPipe(FilterTooLongTransform(src_seq_length=200, tgt_seq_length=200))
[2022-03-26 14:36:19,790 INFO] Counters src:38
[2022-03-26 14:36:19,790 INFO] Counters tgt:38


In [5]:
!onmt_train -config config.yaml

[2022-03-26 14:36:44,494 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 14:36:44,494 INFO] Parsed 2 corpora from -data.
[2022-03-26 14:36:44,494 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-26 14:36:44,495 INFO] Loading vocab from text file...
[2022-03-26 14:36:44,495 INFO] Loading src vocabulary from experiment_1/run/example.vocab.src
[2022-03-26 14:36:44,495 INFO] Loaded src vocab has 38 tokens.
[2022-03-26 14:36:44,495 INFO] Loading tgt vocabulary from experiment_1/run/example.vocab.tgt
[2022-03-26 14:36:44,495 INFO] Loaded tgt vocab has 38 tokens.
[2022-03-26 14:36:44,495 INFO] Building fields with vocab in counters...
[2022-03-26 14:36:44,495 INFO]  * tgt vocab size: 42.
[2022-03-26 14:36:44,496 INFO]  * src vocab size: 40.
[2022-03-26 14:36:44,496 INFO]  * src vocab size = 40
[2022-03-26 14:36:44,496 INFO]  * tgt vocab size = 42
[2022-03-26 14:36:44,498 INFO] Building model...
[2022-03-26 14:36:53,849

In [7]:
!onmt_translate -model experiment_1/run/model_step_7000.pt -src experiment_1/src_test.txt -output experiment_1/test-decoded.txt -gpu 0 -verbose -beam_size 12 

[2022-03-26 14:56:13,990 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-03-26 14:56:14,171 INFO] 
SENT 1: ['r', '@', 'f', 'j', 'u', ':', 't']
PRED 1: r @ f j u : t I d
PRED SCORE: -7.0424

[2022-03-26 14:56:14,171 INFO] 
SENT 2: ['h', 'A', ':', 'd', 'n', ',']
PRED 2: h A : d n , d
PRED SCORE: -5.1942

[2022-03-26 14:56:14,171 INFO] 
SENT 3: ['s', 'p', 'V', 't', '@', 'r', '*']
PRED 3: s p V t @ d
PRED SCORE: -4.8909

[2022-03-26 14:56:14,171 INFO] 
SENT 4: ['f', '@', 'm', 'I', 'l', 'I', '@', 'r', 'a', 'I', 'z'

In [8]:
# Move your data to your Google Drive
%cd ..
!mv /content/experiment_1/ /content/drive/MyDrive/

/
